In [2]:
from pymongo import MongoClient
import json

In [3]:
#Connecting to database 
try:
    conn = MongoClient()
    print("Connected successfully")
except:  
    print("Could not connect to MongoDB")
  
# database
db = conn.database
collection = db.tweets_data

Connected successfully


In [5]:
with open("../json_files/sample_json_file.json", "r") as f:
    data = json.load(f)

In [6]:
keys = ['id_str', 'text', 'created_at', 'entities', 'retweet_count', 'favorite_count', 'lang']

In [12]:
index['id']

1249403769193779202

In [14]:
def mongo_insertor(index, keys):
    """AI is creating summary for mongo_insertor

    Args:
        index ([type]): [description]
        keys ([type]): [description]

    Returns:
        [type]: [description]
    """
    obj = {
        "_id": index['id']
        }
    for key in keys:
        obj[key] = index[key]
    
    obj['user_id'] = index['user']['id']
    return obj


for index in data:
    if 'retweeted_status' in index.keys():
        obj = mongo_insertor(index['retweeted_status'], keys)
        try:
            collection.insert_one(obj)
        except Exception as e:
            print(e)
            pass
        
    if 'quoted_status' in index.keys():
        obj = mongo_insertor(index['quoted_status'], keys)
        try:
            collection.insert_one(obj)
        except Exception as e:
            print(e)
            pass
    
    
    obj = mongo_insertor(index, keys)
    try:
        collection.insert_one(obj)
    except Exception as e:
        print(e)
        pass

DuplicateKeyError: E11000 duplicate key error collection: database.tweets_data index: _id_ dup key: { _id: 1249378751349231616 }